# Tuning and deploy a foundation model


### Learning Objective

This tutorial teaches you how to tune a foundational model on new unseen data and you will use the following Google Cloud products:

- Vertex AI Generative AI Studio
- Vertex AI Pipelines
- Vertex AI Model Registry
- Vertex AI Endpoints

The steps performed include:

- Get training data from BQ and generate a JSONL file
- Upload training data
- Create a pipeline job
- Inspect your model on Vertex AI Model Registry
- Get predictions from your tuned model

Creating an LLM requires massive amounts of data, significant computing resources, and specialized skills. On Vertex AI, tuning allows you to customize a foundation model for more specific tasks or knowledge domains.

While the prompt design is excellent for quick experimentation, if training data is available, you can achieve higher quality by tuning the model. Tuning a model enables you to customize the model response based on examples of the task you want the model to perform.

For more details on tuning have a look at the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

**Quota**: Tuning the text-bison@001  model uses the tpu-v3-8 training resources and the accompanying quotas from your Google Cloud project. Each project has a default quota of eight v3-8 cores, which allows for one to two concurrent tuning jobs. If you want to run more concurrent jobs you need to request additional quota via the [Quotas page](https://console.cloud.google.com/iam-admin/quotas).

**Costs:** This tutorial uses billable a component of Google Cloud `Vertex AI Generative AI Studio`.
Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Setup

In [ ]:
#!pip install sequence-evaluate sentence-transformers rouge --upgrade --user

In [42]:
import random
import string
import time
from typing import Union

import numpy as np
import pandas as pd
from google.cloud import aiplatform, bigquery
from sklearn.model_selection import train_test_split
from vertexai.preview.language_models import TextGenerationModel

In [5]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

BUCKET_NAME = PROJECT_ID
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [6]:
!gsutil ls $BUCKET_URI || gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

gs://dherin-dev/cord19_embeddings.json
gs://dherin-dev/salads.csv
gs://dherin-dev/115851500182/
gs://dherin-dev/7737964263322419200-616112577574862848/
gs://dherin-dev/babyweight/
gs://dherin-dev/babyweight_220707_021136/
gs://dherin-dev/babyweight_220707_021151/
gs://dherin-dev/babyweight_220707_021154/
gs://dherin-dev/car_damage_lab_images/
gs://dherin-dev/classification-bert-20230411003650/
gs://dherin-dev/contextual_bandit_checkpoints/
gs://dherin-dev/covertype/
gs://dherin-dev/models/
gs://dherin-dev/movies/
gs://dherin-dev/staging/
gs://dherin-dev/taxifare-20230710171207/
gs://dherin-dev/taxifare-20230710191151/
gs://dherin-dev/taxifare/


## Tune your Model

Now it's time for you to create a tuning job. Tune a foundation model by creating a pipeline job using Generative AI Studio, cURL, or the Python SDK. In this notebook, we will be using the Python SDK. You will be using a Q&A with a context dataset in JSON format.

### Training Data
💾 Your model tuning dataset must be in a JSONL format where each line contains a single training example. You must make sure that you include instructions.

You will use the StackOverflow data on BigQuery Public Datasets, limiting to questions with the `python` tag, and accepted answers for answers since 2020-01-01.

First create a helper function to let you easily query BigQuery and return the results as a Pandas DataFrame.

In [8]:
def run_bq_query(sql):
    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")

    return df

Next define the query.

In [14]:
query = """
SELECT CONCAT(q.title, q.body) as input_text, a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    10000
"""

In [15]:
df = run_bq_query(query)
df.head()

Finished job_id: a7538c94-f47b-4a06-ac1e-6d76d0b55223


,input_text,output_text
0,How to solve the alpha component ignored in ki...,<p>I don't have a whole picture of your projec...
1,SQLAlchemy with_variant() for MySQL and MariaD...,<p>What happened is you tried to specify two <...
2,Python imap to take multiple functions<p>I am ...,<p>If I understand you correctly you can use <...
3,Django: Table doesn't exist( python manage.py ...,<p>I think what happend is that you lost sync ...
4,split pandas rows into columns and comma separ...,"<p>Use <a href=""https://pandas.pydata.org/docs..."


There should be 10k questions and answers.

In [16]:
print(len(df))

10000


Lets split the data into training and evalation. For Extractive Q&A tasks we advise 100+ training examples. In this case you will use 800.

In [17]:
# split is set to 80/20
train, evaluation = train_test_split(df, test_size=0.2)
print(len(train))

8000


For tuning, the training data first needs to be converted into a JSONL format.

In [26]:
training_data_filename = "tune_data_stack_overflow_python_qa.jsonl"
train.to_json(training_data_filename, orient="records", lines=True)

In [29]:
!head -n 1 $training_data_filename

{"input_text":"How to convert Bytes (UTF-8) embeded emoji in a string<p>I am scraping a data from WhatsApp chat backup (chat.txt). It looks like this :<\/p>\n<pre><code>7\/21\/20, 1:31 PM - mark: Can we look google  \n7\/21\/20, 1:31 PM - elon: No  \n7\/21\/20, 1:31 PM - mark: Can we smile ?  \n7\/21\/20, 1:31 PM - elon: Ya\n<\/code><\/pre>\n<p>While I used line by line extraction<\/p>\n<pre><code>with open ('chat.txt','rb') as file:\n    for line in file:\n        print(str(line.strip()))\n<\/code><\/pre>\n<p>I got this:<\/p>\n<pre><code>b'7\/21\/20, 7:37 AM - mark: Can we look google\\xf0\\x9f\\xa4\\xa9\\xf0\\x9f\\x98\\x82\\xf0\\x9f\\x98\\x82'\nb'7\/21\/20, 7:37 AM - elon: No'\nb'7\/21\/20, 1:31 PM - mark: Can we smile ?'\nb'7\/21\/20, 7:37 AM - elon: Ya\\xf0\\x9f\\x98\\x82'\n<\/code><\/pre>\n<ol>\n<li><p>How can we git rid of <code>b''<\/code> ? ( I tried <code>.decode('utf-8')<\/code>, but it didn't work)<\/p>\n<\/li>\n<li><p>How can I convert<\/p>\n<pre><code>Can we look google\\x

You can then export the local file to GCS, so that it can be used by Vertex AI for the tuning job.

In [30]:
! gsutil cp $training_data_filename $BUCKET_URI

Copying file://tune_data_stack_overflow_python_qa.jsonl [Content-Type=application/octet-stream]...
/ [1 files][ 22.5 MiB/ 22.5 MiB]                                                
Operation completed over 1 objects/22.5 MiB.                                     


You can check to make sure that the file successfully transferred to your Google Cloud Storage bucket:

In [32]:
TRAINING_DATA_URI = f"{BUCKET_URI}/{training_data_filename}"
! gsutil ls -al $TRAINING_DATA_URI

  23637845  2023-09-19T21:11:04Z  gs://dherin-dev/tune_data_stack_overflow_python_qa.jsonl#1695157864253156  metageneration=1
TOTAL: 1 objects, 23637845 bytes (22.54 MiB)


### Model Tuning
Now it's time to start to tune a model. You will use the Vertex AI SDK to submit our tuning job.

#### Recommended Tuning Configurations
✅ Here are some recommended configurations for tuning a foundation model based on the task, in this example Q&A. You can find more in the [documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

Extractive QA:
- Make sure that your train dataset size is 100+
- Training steps [100-500]. You can try more than one value to get the best performance on a particular dataset (e.g. 100, 200, 500)

In [43]:
aiplatform.init(project=PROJECT_ID, location=REGION)

model = TextGenerationModel.from_pretrained("text-bison@001")

Next it's time to start your tuning job. 

**Disclaimer:** tuning and deploying a model takes time.

In [ ]:
TRAIN_STEPS = 100
MODEL_NAME = f"genai-workshop-tuned-model-{time.time()}"
print("Model name:", MODEL_NAME)

model.tune_model(
    training_data=TRAINING_DATA_URI,
    model_display_name=MODEL_NAME,
    train_steps=TRAIN_STEPS,
    # Tuning can only happen in the "europe-west4" location
    tuning_job_location="europe-west4",
    # Model can only be deployed in the "us-central1" location
    tuned_model_location="us-central1",
)

Model name: genai-workshop-tuned-model-1695160329.9899337
Creating PipelineJob
PipelineJob created. Resource name: projects/115851500182/locations/europe-west4/pipelineJobs/tune-large-model-20230919215211
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/115851500182/locations/europe-west4/pipelineJobs/tune-large-model-20230919215211')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/europe-west4/pipelines/runs/tune-large-model-20230919215211?project=115851500182
PipelineJob projects/115851500182/locations/europe-west4/pipelineJobs/tune-large-model-20230919215211 current state:
PipelineState.PIPELINE_STATE_PENDING
PipelineJob projects/115851500182/locations/europe-west4/pipelineJobs/tune-large-model-20230919215211 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/115851500182/locations/europe-west4/pipelineJobs/tune-large-model-20230919215211 current state:
PipelineState.PIPELINE_STATE_RUNNING
P

In [ ]:
print(
    model.predict(
        "Can you provide me with a Python implementation of BERT with Tensorflow? Example: "
    )
)

Following the link above, you can view your pipeline run. As you can see in the screenshot below, it will execute the following steps:

- Validation
- Export managed dataset
- Convert JSONL to TFRecord
- Large language model tuning
- Upload LLM Model

## View your tuned foundational model on Vertex AI Model registry
When your tuning job is finished, your model will be available on Vertex AI Model Registry. The following Python SDK sample shows you how to list tuned models.

In [ ]:
model.list_tuned_model_names()

You can also use the Google Cloud Console UI to view all of your model in [Vertex AI Model Registry](https://console.cloud.google.com/vertex-ai/models?e=13802955&jsmode=O&mods=-ai_platform_fake_service&project=cloud-llm-preview1). Below you can see an example of a tuned foundational model available on Vertex AI Model Registry.

## Use your tuned model to get predictions
Now it's time to get predictions. First you need to get the latest tuned model from the Vertex AI Model registry.

In [ ]:
def fetch_model(project_id, location):
    aiplatform.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained("text-bison@001")
    list_tuned_models = model.list_tuned_model_names()
    tuned_model = list_tuned_models[0]

    return tuned_model

In [ ]:
deployed_model = fetch_model(PROJECT_ID, REGION)
deployed_model = TextGenerationModel.get_tuned_model(deployed_model)

Now you can start send a prompt to the API. Feel free to update the following prompt.

In [ ]:
PROMPT = """
How can I store my TensorFlow checkpoint on Google Cloud Storage?

Python example:

"""

In [ ]:
print(deployed_model.predict(PROMPT))

## Evaulation
It's essential to evaluate your model to understand its performance. Evaluation can be done in an automated way using evaluation metrics like F1 or Rouge. You can also leverage human evaluation methods. Human evaluation methods involve asking humans to rate the quality of the LLM's answers. This can be done through crowdsourcing or by having experts evaluate the responses. Some standard human evaluation metrics include fluency, coherence, relevance, and informativeness. Often you want to choose a mix of evaluation metrics to get a good understanding of your model performance. Below you will find an example of how you can do the evaluation.

In this example you will be using [sequence-evaluate](https://pypi.org/project/sequence-evaluate/) to evaluation the tuned model.

In [ ]:
from seq_eval import SeqEval

evaluator = SeqEval()

Earlier in the notebook, you created a train and eval dataset. Now it's time to take some of the eval data. You will use the questions to get a response from our tuned model, and the answers we will use as a reference:

- **Candidates**: Answers generated by the tuned model.
- **References**: Original answers that we will use to compare.

In [ ]:
evaluation = evaluation.head(
    10
)  # you can change the number of rows you want to use
evaluation_question = evaluation["input_text"]
evaluation_answer = evaluation["output_text"]

Now you can go ahead and generate candidates using the tuned model based on the questions you took from the eval dataset.

In [ ]:
candidates = []

for i in evaluation_question:
    response = deployed_model.predict(i)
    candidates.append(response.text)

len(candidates)

You will also have to create a list of our references. These will you use to evaluate the model's performance.

In [ ]:
references = evaluation_answer.tolist()

len(references)

Next you will generate the evaluation metrics. `evaluator.evaluate` will return a few eval metrics. Some of the important ones are:
- [Blue](https://en.wikipedia.org/wiki/BLEU): The BLEU evaluation metric is a measure of the similarity between a machine-generated text and a human-written reference text.
- [Rouge](https://en.wikipedia.org/wiki/ROUGE_(metric)): The ROUGE evaluation metric is a measure of the overlap between a machine-generated text and a human-written reference text.

In [ ]:
scores = evaluator.evaluate(candidates, references, verbose=False)
print(scores)

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.